In [2]:
from google.cloud import bigquery

client = bigquery.Client()

dataset_ref = client.dataset("nhtsa_traffic_fatalities", project = "bigquery-public-data")

dataset = client.get_dataset(dataset_ref)


Using Kaggle's public dataset BigQuery integration.


In [3]:
# list tables
tables = list(client.list_tables(dataset))

for table in tables:
    print(table.table_id)
    

 accident_2015
 accident_2016
 accident_2017
 accident_2018
 accident_2019
 accident_2020
 cevent_2015
 cevent_2016
 cevent_2017
 cevent_2018
 cevent_2019
 cevent_2020
 damage_2015
 damage_2016
 damage_2017
 damage_2018
 damage_2019
 damage_2020
 distract_2015
 distract_2016
 distract_2017
 distract_2018
 distract_2019
 distract_2020
 drimpair_2015
 drimpair_2016
 drimpair_2017
 drimpair_2018
 drimpair_2019
 drimpair_2020
 factor_2015
 factor_2016
 factor_2017
 factor_2018
 factor_2019
 factor_2020
 maneuver_2015
 maneuver_2016
 maneuver_2017
 maneuver_2018
 maneuver_2019
 maneuver_2020
 nmcrash_2015
 nmcrash_2016
 nmcrash_2017
 nmcrash_2018
 nmcrash_2019
 nmcrash_2020
 nmimpair_2015
 nmimpair_2016
 nmimpair_2017
 nmimpair_2018
 nmimpair_2019
 nmimpair_2020
 nmprior_2015
 nmprior_2016
 nmprior_2017
 nmprior_2018
 nmprior_2019
 nmprior_2020
 parkwork_2015
 parkwork_2016
 parkwork_2017
 parkwork_2018
 parkwork_2019
 parkwork_2020
 pbtype_2015
 pbtype_2016
 pbtype_2017
 pbtype_2018
 pbtyp

In [4]:
table_ref = dataset_ref.table("accident_2015")

table = client.get_table(table_ref)

client.list_rows(table, max_results=5).to_dataframe()

,state_number,state_name,consecutive_number,number_of_vehicle_forms_submitted_all,number_of_motor_vehicles_in_transport_mvit,number_of_parked_working_vehicles,number_of_forms_submitted_for_persons_not_in_motor_vehicles,number_of_persons_not_in_motor_vehicles_in_transport_mvit,number_of_persons_in_motor_vehicles_in_transport_mvit,number_of_forms_submitted_for_persons_in_motor_vehicles,...,minute_of_ems_arrival_at_hospital,related_factors_crash_level_1,related_factors_crash_level_1_name,related_factors_crash_level_2,related_factors_crash_level_2_name,related_factors_crash_level_3,related_factors_crash_level_3_name,number_of_fatalities,number_of_drunk_drivers,timestamp_of_crash
0,30,Montana,300019,5,5,0,0,0,7,7,...,45,0,None,0,None,0,None,1,0,2015-03-28 14:58:00+00:00
1,39,Ohio,390099,7,7,0,0,0,15,15,...,24,27,Backup Due to Prior Crash,0,None,0,None,1,0,2015-02-14 11:19:00+00:00
2,49,Utah,490123,16,16,0,0,0,28,28,...,99,0,None,0,None,0,None,1,0,2015-04-14 12:24:00+00:00
3,48,Texas,481184,6,5,1,0,5,5,10,...,99,0,None,0,None,0,None,1,0,2015-05-27 16:40:00+00:00
4,41,Oregon,410333,11,11,0,0,0,14,14,...,99,0,None,0,None,0,None,1,0,2015-11-17 18:17:00+00:00


In [5]:
# Query to find out the number of accidents for each day of the week
query = """
        SELECT COUNT(consecutive_number) AS num_accidents,
               EXTRACT(DAYOFWEEK FROM timestamp_of_crash) AS day_of_week
        FROM `bigquery-public-data.nhtsa_traffic_fatalities.accident_2015`
        GROUP BY day_of_week
        ORDER BY num_accidents DESC
        """

In [6]:
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**9)
query_job = client.query(query, job_config = safe_config)
accidents_by_day = query_job.to_dataframe()

accidents_by_day

/usr/local/lib/python3.11/dist-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,num_accidents,day_of_week
0,5659,7
1,5298,1
2,4916,6
3,4460,5
4,4182,4
5,4038,2
6,3985,3


In [7]:
# Use Window Function(“OVER”) to include the info about accident

query2 = """
        SELECT 
            consecutive_number,
            timestamp_of_crash,
            EXTRACT(DAYOFWEEK FROM timestamp_of_crash) AS day_of_week,
            COUNT(*) OVER (PARTITION BY EXTRACT(DAYOFWEEK FROM timestamp_of_crash)) as accidents_on_this_day_of_week
        FROM `bigquery-public-data.nhtsa_traffic_fatalities.accident_2015`
        ORDER BY day_of_week, timestamp_of_crash
        """

safe_config2 = bigquery.QueryJobConfig(maximum_bytes_billed =10**9)
query_job = client.query(query2, job_config= safe_config2)

accidents_by_days2 = query_job.to_dataframe()

accidents_by_days2

/usr/local/lib/python3.11/dist-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,consecutive_number,timestamp_of_crash,day_of_week,accidents_on_this_day_of_week
0,320026,2015-01-04 00:10:00+00:00,1,5298
1,480025,2015-01-04 00:40:00+00:00,1,5298
2,10004,2015-01-04 00:57:00+00:00,1,5298
3,510008,2015-01-04 01:03:00+00:00,1,5298
4,120024,2015-01-04 01:20:00+00:00,1,5298
...,...,...,...,...
32533,450901,2015-12-26 23:15:00+00:00,7,5659
32534,320293,2015-12-26 23:21:00+00:00,7,5659
32535,62593,2015-12-26 23:26:00+00:00,7,5659
32536,530510,2015-12-26 23:28:00+00:00,7,5659
